## Decision Tree

In [1]:
# Importing the required packages and libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

C:\Users\VIPLab\anaconda3\envs\pytorch_gpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Reading test CSV file directly from Web, and store it in a pandas DataFrame:
test_df=pd.read_csv("D:/Fauzan/Study PhD/Semester 1/Machine Learning/Homework/HW4/spooky-author-identification/spooky-author-identification/test/test.csv")

# Print first 5 rows
test_df.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [3]:
# Find unique value counts of each author
test_df.value_counts()

id       text                                                                                                                                                                                                         
id00008  And then there was 'The Man in the Bell,' a paper by the by, Miss Zenobia, which I cannot sufficiently recommend to your attention.                                                                              1
id18869  "Good God" I whispered, "can you do that for any time?"                                                                                                                                                          1
id18864  "While I improved in speech, I also learned the science of letters as it was taught to the stranger, and this opened before me a wide field for wonder and delight.                                              1
id18863  It is possible indeed it is far more than probable that he was innocent of all participation in the bloody transacti

In [4]:
# Reading train CSV file directly from Web, and store it in a pandas DataFrame:
train_df = pd.read_csv('D:/Fauzan/Study PhD/Semester 1/Machine Learning/Homework/HW4/spooky-author-identification/spooky-author-identification/train/train.csv')

# Print first 5 rows
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [5]:
# Find unique value counts of each author
train_df['author'].value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

In [6]:
# Split into features and labels
X_train = train_df['text']
y_train = train_df['author']
X_test = test_df['text']

In [7]:
# Transform text data into vectors to run it through machine learning model
vectorizer = CountVectorizer(stop_words='english')
corpus = pd.concat([train_df['text'], test_df['text']])
vectorizer.fit(corpus)

CountVectorizer(stop_words='english')

In [8]:
# Randomly splitting the original dataset into training set and testing set
X_tr, X_tt, y_tr, y_tt = train_test_split(X_train, y_train, test_size=0.3, random_state=2)

X_tr_new = vectorizer.transform(X_tr)
X_tt_new = vectorizer.transform(X_tt)

### Experiment 1

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold

param_distribution = {
    'max_depth': np.arange(1, 15),
}

scoring = {    
    'Accuracy': make_scorer(accuracy_score),
    'F1_Score': make_scorer(fbeta_score, beta=1, average='micro'),    
}

In [11]:
result = []
kf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2)
for fold, (train_index, test_index) in enumerate(kf.split(X_tr, y_tr)):
    X_tr, X_tst = X_tr_new[train_index], X_tr_new[test_index]
    y_tr, y_tst = y_tr[train_index], y_tr[test_index]

    for i in range(1, 20):
        # train
        pca = PCA(i)
        X_t = pca.fit_transform(X_tr)
        search_cv = RandomizedSearchCV(DecisionTreeClassifier(), param_distribution,
                                       scoring=scoring, n_jobs=-1, 
                                       cv=RepeatedStratifiedKFold(n_splits=2, n_repeats=2), 
                                       refit='F1_Score') 
        search_cv.fit(X_t, y_tr)
        model = search_cv.best_estimator_        

        # test
        X_t = pca.transform(X_tst)
        y_pred = model.predict(X_t)

        # model evaluation
        f1 = fbeta_score(y_tst, y_pred, beta=1)
        acc = accuracy_score(y_tst, y_pred)
        
        print(f"fold: {fold} - cp:{i} train: {search_cv.best_score_} test: f1={f1}, acc={acc}")

        result.append((fold, i, acc, f1, pca, model))

KeyError: '[2, 4, 5, 8, 19, 31, 33, 49, 58, 59, 72, 77, 86, 87, 88, 94, 106, 113, 127, 135, 141, 145, 148, 152, 166, 167, 175, 176, 193, 202, 203, 208, 228, 234, 239, 241, 245, 261, 275, 278, 282, 290, 291, 301, 319, 324, 337, 351, 363, 364, 367, 371, 379, 383, 395, 405, 424, 435, 436, 441, 447, 454, 463, 464, 466, 473, 477, 484, 487, 502, 504, 517, 519, 524, 529, 548, 551, 554, 559, 562, 565, 570, 577, 582, 590, 594, 595, 608, 609, 612, 616, 625, 626, 629, 639, 641, 648, 652, 662, 676, 678, 682, 683, 684, 689, 692, 699, 712, 723, 727, 734, 736, 737, 741, 742, 744, 749, 761, 763, 781, 783, 786, 801, 807, 810, 811, 816, 821, 824, 825, 840, 844, 849, 851, 862, 866, 874, 878, 879, 884, 909, 911, 924, 932, 943, 945, 946, 948, 959, 966, 970, 975, 980, 986, 998, 999, 1001, 1006, 1012, 1013, 1022, 1033, 1035, 1037, 1040, 1048, 1053, 1061, 1069, 1071, 1072, 1082, 1093, 1116, 1118, 1128, 1140, 1142, 1147, 1151, 1156, 1157, 1169, 1172, 1185, 1189, 1190, 1195, 1196, 1198, 1204, 1205, 1210, 1213, 

In [ ]:
# "my_decisiontree" is instantiated as an "object" of DecisionTreeClassifier "class" 
dt = DecisionTreeClassifier()
dt.fit(X_tr_new, y_tr)

In [ ]:
# Predicting the Binary Label:
y_predict_dt = dt.predict(X_tt_new)
print(y_predict_dt)

In [ ]:
# Estimating the probability (likelihood) of Each Label: 
y_pred_proba_dt = dt.predict_proba(X_tt_new)
y_pred_proba_dt

In [ ]:
# Find accuracy score
score_dt = accuracy_score(y_tt, y_predict_dt)
print("Decision tree accuracy:", score_dt)

### Experiment 2: Hyperparameter Tuning from GridSearch

In [ ]:
# Creating a dictionary of parameters to use in GridSearchCV
from sklearn.model_selection import GridSearchCV

params = {
    'criterion':  ['gini', 'entropy'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
    'splitter': ['best', 'random']
}

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=5,
    verbose=1,
)

clf.fit(X_tr_new, y_tr)
print(clf.best_params_)

In [ ]:
# Predicting the Binary Label:
y_predict_dt_gs = clf.predict(X_tt_new)
print(y_predict_dt_gs)

In [ ]:
# Find accuracy score
score_dt_gs1 = accuracy_score(y_tt, y_predict_dt_gs)
print("Decision tree accuracy:", score_dt_gs1)

### Experiment 3: Hyperparameter Tuning from GridSearch with Certain Parameters

In [ ]:
# Using the Parameters from GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf_gs2 = DecisionTreeClassifier(max_depth=4, criterion='entropy', max_features=0.6, splitter='best')
clf_gs2.fit(X_tr_new, y_tr)

In [ ]:
predictions = clf_gs2.predict(X_tt_new)
print(y_predict_dt_gs)

In [ ]:
score_dt_gs2 = accuracy_score(y_tt, predictions)
print("Decision tree accuracy:", score_dt_gs2)